In [1]:
import pandas as pd

In [31]:
PATH = '/workspaces/invenlytics/data/bronze_layer/dynamic_supply_chain_logistics_dataset.csv'
df = pd.read_csv(PATH)
# df.sort_values('Scheduled_Delivery', inplace=True)

In [ ]:
df

In [ ]:
import pandas as pd

# Assuming your data is loaded into a DataFrame named 'df'
# Ensure the DataFrame is sorted by timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is a datetime object
df = df.sort_values(by='timestamp').reset_index(drop=True)

# Define the split ratio
train_ratio = 0.9

# Calculate the split index
split_index = int(len(df) * train_ratio)

# Split the dataset
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

# Print the number of rows in each set
print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

# Optional: Save train and test sets to separate files
train_df.to_csv("/workspaces/invenlytics/data/bronze_layer/train_data.csv", index=False)
test_df.to_csv("/workspaces/invenlytics/data/bronze_layer/test_data.csv", index=False)


# Forecast

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [16]:
data = pd.read_csv("/workspaces/invenlytics/data/bronze_layer/test_data.csv")
data

,timestamp,vehicle_gps_latitude,vehicle_gps_longitude,fuel_consumption_rate,eta_variation_hours,traffic_congestion_level,warehouse_inventory_level,loading_unloading_time,handling_equipment_availability,order_fulfillment_status,...,iot_temperature,cargo_condition_status,route_risk_level,customs_clearance_time,driver_behavior_score,fatigue_monitoring_score,disruption_likelihood_score,delay_probability,risk_classification,delivery_time_deviation
0,2024-04-17 10:00:00,46.163528,-119.606918,5.718954,-0.809716,7.719687,758.495200,0.500022,0.002687,0.931082,...,-9.999971,0.650040,8.997106,0.863728,0.593180,0.636521,0.607316,0.984113,Moderate Risk,9.988801
1,2024-04-17 11:00:00,37.871733,-118.009036,7.309518,2.083823,8.101871,789.795234,2.272132,0.278724,0.865180,...,-6.267043,0.154350,9.606195,0.500233,0.015645,0.050199,0.184976,0.763337,Low Risk,-0.998835
2,2024-04-17 12:00:00,43.002874,-71.578048,5.000488,-1.730852,7.566148,121.530139,4.752455,0.555171,0.384586,...,-10.000000,0.397618,5.706199,2.363669,0.838864,0.951450,0.639941,0.996785,Moderate Risk,9.788209
3,2024-04-17 13:00:00,41.596057,-92.745829,10.995314,4.976114,3.751966,962.958292,4.989534,0.763430,0.267808,...,-9.984718,0.036936,8.601600,4.753376,0.922709,0.921782,0.999968,0.115914,High Risk,7.981625
4,2024-04-17 14:00:00,44.721542,-70.511692,5.376873,3.622238,3.727591,5.217055,4.493025,0.616115,0.987126,...,-8.757466,0.037010,1.067124,3.869749,0.326371,0.736773,0.587341,0.992351,Moderate Risk,5.361576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,2024-08-28 20:00:00,48.496665,-104.031158,5.421806,4.394370,0.192421,127.684977,3.607803,0.208643,0.992997,...,-8.835069,0.199377,9.893402,0.966050,0.109571,0.511074,0.953467,0.916745,High Risk,5.506049
3203,2024-08-28 21:00:00,34.670137,-89.893091,10.624778,2.700635,8.736143,84.638889,1.218165,0.472389,0.946031,...,-2.517654,0.673775,0.854378,1.665889,0.530289,0.868418,0.995329,0.808963,High Risk,-1.876938
3204,2024-08-28 22:00:00,42.963338,-82.149873,5.071596,0.485288,7.420200,48.500920,4.999149,0.064366,0.437687,...,29.410878,0.949644,4.480552,0.690990,0.537100,0.896798,0.442299,0.959197,Moderate Risk,9.790219
3205,2024-08-28 23:00:00,32.082410,-70.070154,5.000000,4.759411,0.932050,972.437425,0.505242,0.250949,0.517051,...,-6.951137,0.407377,9.185549,4.462204,0.012896,0.463018,0.999999,0.557340,High Risk,9.956879


In [18]:
data['disruption_likelihood_score'] > 0.5

0        True
1       False
2        True
3        True
4        True
        ...  
3202     True
3203     True
3204    False
3205     True
3206     True
Name: disruption_likelihood_score, Length: 3207, dtype: bool

In [4]:


def augment_dataset(input_file, total_rows=100000):
    """
    Load a dataset, augment it with new columns, and return the updated DataFrame.

    Parameters:
        input_file (str): Path to the existing dataset (CSV format).
        total_rows (int): Total number of rows required in the dataset (not used).

    Returns:
        pd.DataFrame: The augmented dataset.
    """
    try:
        # Load existing dataset
        existing_df = pd.read_csv(input_file)

        # New columns to be added
        suppliers = ['Alibaba', 'H&M', 'IKEA', 'Wrogn']
        delivery_modes = ['Air', 'Sea', 'Road', 'Rail']
        disruption_types = ['Weather', 'Supplier Issue', 'Logistics', 'Geopolitical', 'None']
        regions = ['North America', 'Europe', 'Asia', 'South America']
        weather_conditions = ['Sunny', 'Rainy', 'Snowy', 'Cloudy']

        # Add new columns for existing rows if they don't exist
        if 'Supplier' not in existing_df.columns:
            existing_df['Supplier'] = np.random.choice(suppliers, len(existing_df))
        if 'Region' not in existing_df.columns:
            existing_df['Region'] = np.random.choice(regions, len(existing_df))
        if 'Delivery_Mode' not in existing_df.columns:
            existing_df['Delivery_Mode'] = np.random.choice(delivery_modes, len(existing_df))
        if 'Disruption_Type' not in existing_df.columns:
            existing_df['Disruption_Type'] = np.random.choice(disruption_types, len(existing_df))
        if 'Weather_Conditions' not in existing_df.columns:
            existing_df['Weather_Conditions'] = np.random.choice(weather_conditions, len(existing_df))
        if 'Scheduled_Delivery' not in existing_df.columns:
            existing_df['Scheduled_Delivery'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(1, 10, len(existing_df)), unit='D')
        if 'Actual_Delivery' not in existing_df.columns:
            existing_df['Actual_Delivery'] = existing_df['Scheduled_Delivery'] + pd.to_timedelta(np.random.randint(-5, 15, len(existing_df)), unit='D')
        if 'Freight_Cost' not in existing_df.columns:
            existing_df['Freight_Cost'] = np.round(np.random.uniform(100, 1000, len(existing_df)), 2)

        updated_df = existing_df.dropna()

        return updated_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
# input_csv_path = "path/to/your/input.csv"
# result_df = augment_dataset(input_file=input_csv_path)


In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta

def augment_dataset(input_file):
    """
    Load a dataset, augment it with new columns, and return the updated DataFrame.

    Parameters:
        input_file (str): Path to the existing dataset (CSV format).
        total_rows (int): Total number of rows required in the dataset (not used).

    Returns:
        pd.DataFrame: The augmented dataset.
    """
    try:
        # Load existing dataset
        existing_df = pd.read_csv(input_file)

        # New columns to be added
        suppliers = ['Alibaba', 'H&M', 'IKEA', 'Wrogn']
        delivery_modes = ['Air', 'Sea', 'Road', 'Rail']
        disruption_types = ['Weather', 'Supplier Issue', 'Logistics', 'Geopolitical', 'None']
        regions = ['North America', 'Europe', 'Asia', 'South America']
        weather_conditions = ['Sunny', 'Rainy', 'Snowy', 'Cloudy']

        # Add new columns for existing rows if they don't exist
        if 'Supplier' not in existing_df.columns:
            existing_df['Supplier'] = np.random.choice(suppliers, len(existing_df))
        if 'Region' not in existing_df.columns:
            existing_df['Region'] = np.random.choice(regions, len(existing_df))
        if 'Delivery_Mode' not in existing_df.columns:
            existing_df['Delivery_Mode'] = np.random.choice(delivery_modes, len(existing_df))
        if 'Disruption_Type' not in existing_df.columns:
            existing_df['Disruption_Type'] = np.random.choice(disruption_types, len(existing_df))
        if 'Weather_Conditions' not in existing_df.columns:
            existing_df['Weather_Conditions'] = np.random.choice(weather_conditions, len(existing_df))
        if 'Scheduled_Delivery' not in existing_df.columns:
            existing_df['Scheduled_Delivery'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(1, 10, len(existing_df)), unit='D')
        if 'Actual_Delivery' not in existing_df.columns:
            existing_df['Actual_Delivery'] = existing_df['Scheduled_Delivery'] + pd.to_timedelta(np.random.randint(-5, 15, len(existing_df)), unit='D')
        if 'Freight_Cost' not in existing_df.columns:
            existing_df['Freight_Cost'] = np.round(np.random.uniform(100, 1000, len(existing_df)), 2)

        updated_df = existing_df.dropna()
        updated_df.sort_values('timestamp', inplace=True)

        return updated_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
# input_csv_path = "path/to/your/input.csv"
# result_df = augment_dataset(input_file=input_csv_path)


In [6]:
raw_df = augment_dataset('/workspaces/invenlytics/data/bronze_layer/test_data.csv')

In [7]:
import logging
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from pandas.tseries.holiday import USFederalHolidayCalendar
import pandas as pd

def preprocess_features(data, selected_columns):
    """
    Preprocess the dataset by performing target encoding, categorical encoding, 
    and feature engineering (rolling, lag, expanding window features).

    Parameters:
    data (pd.DataFrame): The input DataFrame.
    selected_columns (list): List of columns to select from the dataset.

    Returns:
    pd.DataFrame: The processed dataset with new features.
    """
    try:
        logging.info("Starting preprocessing pipeline.")

        data = data[selected_columns]
        logging.info(f"Dataset loaded successfully. {data.info()}")

        # Target encoding for 'Disruption_Type'
        if "Disruption_Type" in data.columns:
            data["Disruption"] = data["Disruption_Type"].apply(lambda x: 0 if x == "None" else 1)
        logging.info("Target encoding completed.")

        # Encode categorical features
        encoder = LabelEncoder()
        for col in ["Region", "Delivery_Mode", "Supplier"]:
            if col in data.columns:
                data[col] = encoder.fit_transform(data[col])

        # One-hot encoding for Weather_Conditions
        if "Weather_Conditions" in data.columns:
            data = pd.get_dummies(data, columns=["Weather_Conditions"], drop_first=True)
        logging.info("Categorical encoding completed.")

        # Generate holiday-related features
        holiday_calendar = USFederalHolidayCalendar()
        holidays = holiday_calendar.holidays(
            start=data["Scheduled_Delivery"].min(), 
            end=data["Scheduled_Delivery"].max()
        )

        data["Is_Business_Day"] = data["Scheduled_Delivery"].dt.dayofweek < 5
        data["Is_Holiday"] = data["Scheduled_Delivery"].dt.date.isin(holidays.date)
        data["Is_Working_Day"] = data["Is_Business_Day"] & ~data["Is_Holiday"]
        data["Week_Of_Year"] = data["Scheduled_Delivery"].dt.isocalendar().week
        data["Quarter"] = data["Scheduled_Delivery"].dt.quarter
        data["Is_Weekend"] = ~data["Is_Business_Day"]
        logging.info("Holiday-related features created.")

        # Generate time-based features
        if "Actual_Delivery" in data.columns:
            data["Delivery_Duration"] = (data["Actual_Delivery"] - data["Scheduled_Delivery"]).dt.total_seconds() / 3600

        # Normalize selected numerical features
        normalize_cols = [
            "traffic_congestion_level", "port_congestion_level", "weather_condition_severity", 
            "fuel_consumption_rate", "driver_behavior_score", "fatigue_monitoring_score"
        ]
        scaler = MinMaxScaler()
        for col in normalize_cols:
            if col in data.columns:
                data[col] = scaler.fit_transform(data[[col]])
        logging.info("Normalization completed.")

        # Define rolling, lag, and expanding window features
        rolling_cols = ["traffic_congestion_level", "port_congestion_level", "supplier_reliability_score"]
        lag_cols = rolling_cols
        window_sizes = [3, 7, 14, 24, 48]
        lag_steps = [1, 2, 3, 4, 5, 6, 7, 14, 24, 48]

        # Generate rolling and expanding features
        for window in window_sizes:
            for col in rolling_cols:
                if col in data.columns:
                    data[f"{col}_rolling_mean_{window}"] = data[col].rolling(window=window).mean()
                    data[f"{col}_rolling_std_{window}"] = data[col].rolling(window=window).std()

        for col in rolling_cols:
            if col in data.columns:
                data[f"{col}_expanding_mean"] = data[col].expanding(min_periods=7).mean()
        logging.info("Rolling and expanding features generated.")

        # Generate lag features
        for col in lag_cols:
            if col in data.columns:
                for lag in lag_steps:
                    data[f"{col}_lag_{lag}"] = data[col].shift(lag)
        logging.info("Lag features generated.")

        # Feature interactions
        if "traffic_congestion_level" in data.columns and "port_congestion_level" in data.columns:
            data["Traffic_Port_Interaction"] = data["traffic_congestion_level"] * data["port_congestion_level"]

        # Drop rows with NaN values introduced by feature generation
        data.dropna(inplace=True)
       
        logging.info("Dropped NaN values. Preprocessing complete.")

        return data

    except Exception as e:
        logging.error(f"An error occurred during preprocessing: {e}", exc_info=True)
        raise


In [8]:
selected_columns = [
            "Scheduled_Delivery", "Actual_Delivery", "Disruption_Type", "Region", "Delivery_Mode", "Supplier",
            "Weather_Conditions",  "traffic_congestion_level", "port_congestion_level", "weather_condition_severity", 
            "fuel_consumption_rate", "driver_behavior_score", "fatigue_monitoring_score", 'supplier_reliability_score'
]
features = preprocess_features(data=raw_df, selected_columns=selected_columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Scheduled_Delivery          3207 non-null   datetime64[ns]
 1   Actual_Delivery             3207 non-null   datetime64[ns]
 2   Disruption_Type             3207 non-null   object        
 3   Region                      3207 non-null   object        
 4   Delivery_Mode               3207 non-null   object        
 5   Supplier                    3207 non-null   object        
 6   Weather_Conditions          3207 non-null   object        
 7   traffic_congestion_level    3207 non-null   float64       
 8   port_congestion_level       3207 non-null   float64       
 9   weather_condition_severity  3207 non-null   float64       
 10  fuel_consumption_rate       3207 non-null   float64       
 11  driver_behavior_score       3207 non-null   float64     

/tmp/ipykernel_24362/1629018701.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Disruption"] = data["Disruption_Type"].apply(lambda x: 0 if x == "None" else 1)
/tmp/ipykernel_24362/1629018701.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = encoder.fit_transform(data[col])
/tmp/ipykernel_24362/1629018701.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [9]:
features.head().T

,48,49,50,51,52
Scheduled_Delivery,2024-01-09 00:00:00,2024-01-09 00:00:00,2024-01-05 00:00:00,2024-01-02 00:00:00,2024-01-07 00:00:00
Actual_Delivery,2024-01-21 00:00:00,2024-01-20 00:00:00,2024-01-02 00:00:00,2023-12-31 00:00:00,2024-01-04 00:00:00
Disruption_Type,Supplier Issue,Geopolitical,Supplier Issue,Weather,None
Region,3,1,3,3,0
Delivery_Mode,2,3,3,0,3
...,...,...,...,...,...
supplier_reliability_score_lag_7,0.863751,0.710375,0.952334,0.994503,0.506392
supplier_reliability_score_lag_14,0.836355,0.457293,0.019301,0.287509,0.949191
supplier_reliability_score_lag_24,0.919903,0.449983,0.010065,0.984601,0.869886
supplier_reliability_score_lag_48,0.072445,0.525464,0.804724,0.595401,0.875779


# Model

In [10]:
import mlflow
import pandas as pd
import logging
from pipelines import feature_pipeline
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_model(logged_model_path):
    """
    Load the MLflow model from the given path.
    """
    logger.info("Loading model from MLflow...")
    try:
        model = mlflow.pyfunc.load_model(logged_model_path)
        logger.info("Model loaded successfully.")
        return model
    except Exception as e:
        logger.error(f"Failed to load the model. Error: {e}")
        raise

def run_inference(processed_df, logged_model_path, pipeline=None):
    """
    Perform inference using the given model and data.
    """
    try:
        model = load_model(logged_model_path)
        expected_columns = model.metadata.get_input_schema().input_names()
        processed_df.drop(columns=['Disruption_Type', 'Disruption', 'Disruption_Type'], inplace=True)
        logger.info("Inferencing...")
        processed_df['Disruption_Forecast'] = model.predict(processed_df) < 0.5
        logger.info("Inference completed.")
        return processed_df
    except Exception as e:
        logger.error(f"An error occurred during inference. Error: {e}")
        raise

In [11]:
model = load_model('runs:/77eb38c47111407e98cb2e02fcfad5be/model')

In [12]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.lightgbm
  run_id: 77eb38c47111407e98cb2e02fcfad5be

In [13]:
# file:///workspaces/invenlytics/mlruns/439170859911534597/baabb1cd86164ebca7d67e77bf9c684e/artifacts/model

In [15]:
run_inference(processed_df=features, logged_model_path='file:///workspaces/invenlytics/mlruns/439170859911534597/9576b38ce6664d028498589e3589cd70/artifacts/model')

2025/01/10 06:52:51 WARNING mlflow.models.utils: Found extra inputs in the model input that are not defined in the model signature: `['Actual_Delivery', 'Scheduled_Delivery']`. These inputs will be ignored.


,Scheduled_Delivery,Actual_Delivery,Region,Delivery_Mode,Supplier,traffic_congestion_level,port_congestion_level,weather_condition_severity,fuel_consumption_rate,driver_behavior_score,...,supplier_reliability_score_lag_3,supplier_reliability_score_lag_4,supplier_reliability_score_lag_5,supplier_reliability_score_lag_6,supplier_reliability_score_lag_7,supplier_reliability_score_lag_14,supplier_reliability_score_lag_24,supplier_reliability_score_lag_48,Traffic_Port_Interaction,Disruption_Forecast
48,2024-01-09,2024-01-21,3,2,3,0.598741,0.867988,0.023881,5.273708e-01,0.294660,...,0.506392,0.994503,0.952334,0.710375,0.863751,0.836355,0.919903,0.072445,0.519700,False
49,2024-01-09,2024-01-20,1,3,2,0.018184,0.935327,0.246202,1.292248e-01,0.361146,...,0.891195,0.506392,0.994503,0.952334,0.710375,0.457293,0.449983,0.525464,0.017008,False
50,2024-01-05,2024-01-02,3,3,2,0.267301,0.999828,0.533386,6.589606e-01,0.055061,...,0.491554,0.891195,0.506392,0.994503,0.952334,0.019301,0.010065,0.804724,0.267255,False
51,2024-01-02,2023-12-31,3,0,3,0.109003,0.078928,0.931780,6.315210e-01,0.934158,...,0.987237,0.491554,0.891195,0.506392,0.994503,0.287509,0.984601,0.595401,0.008603,False
52,2024-01-07,2024-01-04,0,3,2,0.109393,0.203290,0.970202,3.384619e-01,0.033281,...,0.659482,0.987237,0.491554,0.891195,0.506392,0.949191,0.869886,0.875779,0.022239,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,2024-01-05,2024-01-12,1,2,3,0.019242,0.970083,0.792648,2.812987e-02,0.109571,...,0.570806,0.001259,0.001207,0.010005,0.004973,0.894688,0.702056,0.623301,0.018666,False
3203,2024-01-06,2024-01-01,3,0,3,0.873615,0.275287,0.034155,3.751111e-01,0.530289,...,0.118903,0.570806,0.001259,0.001207,0.010005,0.390854,0.264046,0.981739,0.240495,False
3204,2024-01-02,2024-01-09,1,1,3,0.742021,0.049818,0.022338,4.774674e-03,0.537100,...,0.855789,0.118903,0.570806,0.001259,0.001207,0.724509,0.029885,0.182434,0.036966,False
3205,2024-01-07,2024-01-20,0,2,2,0.093205,0.969746,0.782337,2.120227e-09,0.012896,...,0.316930,0.855789,0.118903,0.570806,0.001259,0.796999,0.887133,0.678591,0.090385,False
